In [ ]:
# Credit Card Fraud Detection: Gözetimli Öğrenme Projesi

## 1. Proje Tanımı

- **Problem:** Kredi kartı işlemlerinin sahte (fraud) olup olmadığını tahmin etmek.  
- **Veri Seti:** [Kaggle - Credit Card Fraud Detection](https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud)  
- **Amaç:** Gerçek hayatta finansal kayıpları önlemek için yeni işlemlerin sahte olup olmadığını tespit eden bir model geliştirmek.

> **Not:** Bonus olarak, aynı veri seti üzerinde k-Means ile temel bir kümeleme de eklendi.

: 

## 2. Kütüphanelerin Yüklenmesi

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.cluster import KMeans

import warnings
warnings.filterwarnings('ignore')

In [ ]:
## 3. Veri Setinin Yüklenmesi

In [ ]:
# Kaggle'dan indirdiğiniz "creditcard.csv" dosyasını aynı dizine koyun
df = pd.read_csv('creditcard.csv')
df.shape

In [ ]:
## 4. Keşifsel Veri Analizi (EDA)

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
**Eksik veri var mı?**

In [ ]:
df.isnull().sum().sum()

In [ ]:
**Sınıf Dağılımı**

In [ ]:
sns.countplot(x='Class', data=df)
plt.title('Sınıf Dağılımı (0=Normal, 1=Fraud)')
plt.show()
print(df['Class'].value_counts(normalize=True))

In [ ]:
Veri ciddi şekilde dengesizdir (%0.17 fraud).

In [ ]:
**Bazı özelliklerin dağılımı:**

In [ ]:
plt.figure(figsize=(14,6))
for i, col in enumerate(['V1', 'V2', 'V3', 'Amount']):
    plt.subplot(1,4,i+1)
    sns.histplot(df[col], bins=30, kde=True)
    plt.title(col)
plt.tight_layout()
plt.show()

In [ ]:
**Korelasyon Matrisi:**

In [ ]:
plt.figure(figsize=(12,9))
sns.heatmap(df.corr(), cmap='coolwarm', vmin=-1, vmax=1, center=0)
plt.title('Korelasyon Matrisi')
plt.show()

In [ ]:
## 5. Veri Ön İşleme

In [ ]:
scaler = StandardScaler()
df['normAmount'] = scaler.fit_transform(df['Amount'].values.reshape(-1, 1))
df = df.drop(['Time','Amount'], axis=1)

In [ ]:
**Eğitim ve test kümeleri:**

In [ ]:
X = df.drop('Class', axis=1)
y = df['Class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42, stratify=y)

In [ ]:
## 6. Model Seçimi & Hiperparametre Optimizasyonu  
- Lojistik Regresyon, Karar Ağacı, KNN ve Random Forest karşılaştırılacak

In [ ]:
models = {
    'Logistic Regression': LogisticRegression(),
    'Decision Tree': DecisionTreeClassifier(),
    'KNN': KNeighborsClassifier(),
    'Random Forest': RandomForestClassifier(n_jobs=-1)
}
results = {}

for name, model in models.items():
    scores = cross_val_score(model, X_train, y_train, cv=5, scoring='roc_auc')
    results[name] = scores.mean()
    print(f"{name}: ROC-AUC = {scores.mean():.4f}")

In [ ]:
**Random Forest en iyi ROC-AUC puanına sahip. Hiperparametre optimizasyonu yapılacak.**

In [ ]:
param_grid = {
    'n_estimators': [50, 100],
    'max_depth': [5, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
}
rf = RandomForestClassifier(n_jobs=-1, random_state=42)
grid = GridSearchCV(rf, param_grid, cv=3, scoring='roc_auc', verbose=1, n_jobs=-1)
grid.fit(X_train, y_train)
print("En iyi parametreler:", grid.best_params_)

In [ ]:
## 7. Model Değerlendirme

In [ ]:
best_rf = grid.best_estimator_
y_pred = best_rf.predict(X_test)
y_prob = best_rf.predict_proba(X_test)[:,1]

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(f"ROC-AUC: {roc_auc_score(y_test, y_prob):.4f}")

In [ ]:
fpr, tpr, _ = roc_curve(y_test, y_prob)
plt.plot(fpr, tpr, label='Random Forest (AUC = %.4f)' % roc_auc_score(y_test, y_prob))
plt.plot([0,1], [0,1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend()
plt.show()

In [ ]:
### Önemli Özellikler

In [ ]:
feat_importances = pd.Series(best_rf.feature_importances_, index=X.columns)
feat_importances.nlargest(10).plot(kind='barh')
plt.title('En Önemli Özellikler')
plt.show()

In [ ]:
## BONUS: Gözetimsiz Öğrenme (k-Means ile Kümeleme)

Yalnızca V1 ve V2 özellikleri kullanıldı (görselleştirme için).

In [ ]:
kmeans = KMeans(n_clusters=2, random_state=42)
subset = df[['V1','V2']].sample(5000, random_state=42)
labels = kmeans.fit_predict(subset)
plt.scatter(subset['V1'], subset['V2'], c=labels, cmap='coolwarm', alpha=0.5)
plt.title('k-Means ile Kümeleme (V1-V2)')
plt.xlabel('V1')
plt.ylabel('V2')
plt.show()